<a href="https://colab.research.google.com/github/cChaucha/sales-prediction-eating/blob/main/Proyecto_1_Parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Continuaremos trabajando con el repositorio GitHub que creaste la semana pasada para tu primer proyecto de portafolio. El objetivo de tus proyectos de portafolio es tener elementos para mostrarle a potenciales empleadores que demuestren tus habilidades y competencias en la ciencia de datos.

El primer proyecto será una predicción de ventas para productos alimenticios que se venden en distintas tiendas. El objetivo es ayudar a la tienda a comprender las propiedades de los productos y establecimientos que tienen un rol crucial en aumentar las ventas.

Para la parte 2, tu tarea es importar los datos y usar Pandas para comenzar a limpiar y explorar los datos. En la ciencia de datos, pasaremos cerca del 80% de nuestro tiempo preparando los datos, ¡así que este es un paso muy importante! Primero, clona el repositorio GitHub que creaste para alojar tu proyecto, para que puedas asignarle este repositorio a tu trabajo. Ve el video al final de esta tarea si no tienes seguridad de cómo hacerlo.

Descarga los datos usando este enlace:  descarga los datos. (Nota: Fuente original de los datos)
Explora los datos: ¿Qué necesitas para limpiar estos datos?
¿Hay valores faltantes en el conjunto de datos? Es posible que algunas tiendas no informen de todos los datos a causa de problemas técnicos u otros problemas. De ser así, trata con ello apropiadamente.
Aquí hay algunos recursos para que consideres maneras más avanzadas para lidiar con valores faltantes en tu conjunto de datos (revisa los recursos Pandas adicionales opcionales al final de este capítulo para más recursos sobre esto):
Datos faltantes, sus tiopos y métodos estadísticos para lidiar con ellos
6 formas diferentes de compensar los valores faltantes en un conjunto de datos
Valores en un conjunto de datos
Tratamiento de datos fdaltantes: Supuestos y métodos clave para el análisis aplicados


¿Cuáles son algunos resúmenes estadísticos, información agregada, u otras tendencias que puedes observar en los datos usando Pandas?
Aquí hay un diccionario de datos para este conjunto de datos:
ombre de la variable	Descripción
* Item_Identifier	Número de identificación único del producto
* Item_Weight	Peso del producto
* Item_Fat_Content	Si el producto es bajo en grasas o regular
* Item_Visibility	El porcentaje de área total de visualización de todos los productos en la tienda asignados a este producto particular
* Item_Type	La categoría a la que pertenece el producto
* Item_MRP	Precio Máximo Minorista (precio de lista) del producto
* Outlet_Identifier	Número de identificación único de la tienda
* Outlet_Establishment_Year	El año en el que se estableció la tienda
* Outlet_Size	El tamaño de la tienda en cuanto a la superficie total que cubre
* Outlet_Location_Type	El tipo de área donde se ubica la tienda
* Outlet_Type	Si la tienda es un almacén o algún tipo de supermercado
* Item_Outlet_Sales	Las ventas del producto en la tienda particular. Esta es la * variable objetivo que se debe predecir.

#Pre-Work

In [2]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
#Import Libraries
import pandas as pd

In [5]:
#Import data
data = "/content/drive/MyDrive/Data Analist/BBDD/sales_predictions.csv"
df = pd.read_csv(data)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


Veremos la variacion de datos faltantes de tipo "Item_Weight" faltantes en función a la variable "Outlet_Type". Esta información nos dará un panorama de qué tipo de tienda no genera informes de inventario.

In [7]:
#Veremos la variacion de cada tienda
weight_by_oulet = df.groupby("Outlet_Type")["Item_Weight"].count()
weight_by_oulet


Outlet_Type
Grocery Store         555
Supermarket Type1    5577
Supermarket Type2     928
Supermarket Type3       0
Name: Item_Weight, dtype: int64

Completaremos los datos faltantes de la columna "Item_Weight" según la identificación de los mismos, ya que al tener mismo "Item_Weight" deducimos que tendrpan mismo peso.

In [8]:
# Paso 1: Agrupar por "Item_Identifier" y obtener el primer valor no nulo de "Item_Weight" en cada grupo
first_non_null_weight_by_identifier = df.groupby("Item_Identifier")["Item_Weight"].first()

# Paso 2: Crear un diccionario que mapee "Item_Identifier" con su primer peso no nulo
identifier_weight_mapping = first_non_null_weight_by_identifier.to_dict()

# Paso 3: Usar el diccionario para completar los valores faltantes en "Item_Weight"
df["Item_Weight"] = df["Item_Weight"].fillna(df["Item_Identifier"].map(identifier_weight_mapping))

In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


Logramos completar casi todos los datos de la columna "Item_Weight", pero aún nos queda algunos por analizar.

In [10]:
grouped_null_rows = df[df['Item_Weight'].isnull()]
grouped_null_rows

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


Se filtrará las filas que tengan como

In [16]:
filtro_FDN = df[(df["Item_Identifier"].str.startswith("FDN")) & (df["Item_Type"] == "Frozen Foods")]
filtro_ordenado = filtro_FDN.sort_values(by="Item_Identifier")

filtro_FDN

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
81,FDN04,11.80,reg,0.014087,Frozen Foods,180.3344,OUT046,1997,Small,Tier 1,Supermarket Type1,1427.4752
530,FDN16,12.60,Regular,0.062648,Frozen Foods,104.8990,OUT013,1987,High,Tier 3,Supermarket Type1,1960.7810
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
1031,FDN16,12.60,Regular,0.062700,Frozen Foods,102.1990,OUT046,1997,Small,Tier 1,Supermarket Type1,1960.7810
1142,FDN04,11.80,Regular,0.014019,Frozen Foods,178.1344,OUT027,1985,Medium,Tier 3,Supermarket Type3,3747.1224
2030,FDN16,12.60,Regular,0.063055,Frozen Foods,102.3990,OUT017,2007,NaN,Tier 2,Supermarket Type1,2270.3780
2093,FDN16,12.60,Regular,0.062798,Frozen Foods,103.9990,OUT049,1999,Medium,Tier 1,Supermarket Type1,825.5920
2105,FDN04,11.80,Regular,0.014075,Frozen Foods,176.8344,OUT013,1987,High,Tier 3,Supermarket Type1,3568.6880
2729,FDN04,11.80,Regular,0.014144,Frozen Foods,180.4344,OUT018,2009,Medium,Tier 3,Supermarket Type2,535.3032
2845,FDN04,11.80,Regular,0.014116,Frozen Foods,178.5344,OUT045,2002,NaN,Tier 2,Supermarket Type1,3390.2536


In [17]:
media = filtro_FDN["Item_Weight"].mean()

# Calcular la mediana de "Item_Weight"
mediana = filtro_FDN["Item_Weight"].median()

# Calcular la moda de "Item_Weight"
moda = filtro_FDN["Item_Weight"].mode()

print("Media:", media)
print("Mediana:", mediana)
print("Moda:", moda)

Media: 9.453333333333333
Mediana: 11.8
Moda: 0    5.88
Name: Item_Weight, dtype: float64


Una vez realizada el análisis, no se va a considerar los datos de la fila con identificación FDN52

In [19]:
df = df.drop(df[df["Item_Identifier"] == "FDN52"].index)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8522 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8522 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8522 non-null   object 
 3   Item_Visibility            8522 non-null   float64
 4   Item_Type                  8522 non-null   object 
 5   Item_MRP                   8522 non-null   float64
 6   Outlet_Identifier          8522 non-null   object 
 7   Outlet_Establishment_Year  8522 non-null   int64  
 8   Outlet_Size                6112 non-null   object 
 9   Outlet_Location_Type       8522 non-null   object 
 10  Outlet_Type                8522 non-null   object 
 11  Item_Outlet_Sales          8522 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.5+ KB


In [20]:
grouped_null_rows = df[df['Item_Weight'].isnull()]
grouped_null_rows

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


Analizaremos la fila del producto con Item_Idenifier == FDK57

In [24]:
filtro_FDK = df[(df["Item_Identifier"].str.startswith("FDK")) & (df["Item_Type"] == "Snack Foods") & (df["Item_Fat_Content"]=="Low Fat")]
filtro_ordenado = filtro_FDK.sort_values(by="Item_Weight")

filtro_FDK

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
55,FDK21,7.905,Low Fat,0.010053,Snack Foods,249.0408,OUT018,2009,Medium,Tier 3,Supermarket Type2,6258.5200
245,FDK21,7.905,Low Fat,0.010010,Snack Foods,249.0408,OUT035,2004,Small,Tier 2,Supermarket Type1,3755.1120
278,FDK21,7.905,Low Fat,0.016759,Snack Foods,250.4408,OUT010,1998,NaN,Tier 3,Grocery Store,500.6816
782,FDK34,13.350,Low Fat,0.038495,Snack Foods,240.2564,OUT013,1987,High,Tier 3,Supermarket Type1,2860.2768
1153,FDK22,9.800,Low Fat,0.026193,Snack Foods,215.6850,OUT018,2009,Medium,Tier 3,Supermarket Type2,649.1550
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
2523,FDK21,7.905,Low Fat,0.010004,Snack Foods,250.7408,OUT013,1987,High,Tier 3,Supermarket Type1,5757.8384
2566,FDK09,15.200,Low Fat,0.091763,Snack Foods,228.5352,OUT046,1997,Small,Tier 1,Supermarket Type1,1832.2816
3000,FDK34,13.350,Low Fat,0.038605,Snack Foods,236.3564,OUT045,2002,NaN,Tier 2,Supermarket Type1,2145.2076
3266,FDK33,17.850,Low Fat,0.011233,Snack Foods,211.5560,OUT035,2004,Small,Tier 2,Supermarket Type1,5752.5120


In [25]:
media = filtro_FDK["Item_Weight"].mean()

# Calcular la mediana de "Item_Weight"
mediana = filtro_FDK["Item_Weight"].median()

# Calcular la moda de "Item_Weight"
moda = filtro_FDK["Item_Weight"].mode()

print("Media:", media)
print("Mediana:", mediana)
print("Moda:", moda)

Media: 12.470757575757574
Mediana: 13.35
Moda: 0    7.905
Name: Item_Weight, dtype: float64


Una vez realizado el análisis, se concluye que se aliminarán las filas que tengan datos nulos en la columna "Item_Weight"

In [26]:
df = df.dropna(subset=["Item_Weight"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8519 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8519 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8519 non-null   object 
 3   Item_Visibility            8519 non-null   float64
 4   Item_Type                  8519 non-null   object 
 5   Item_MRP                   8519 non-null   float64
 6   Outlet_Identifier          8519 non-null   object 
 7   Outlet_Establishment_Year  8519 non-null   int64  
 8   Outlet_Size                6109 non-null   object 
 9   Outlet_Location_Type       8519 non-null   object 
 10  Outlet_Type                8519 non-null   object 
 11  Item_Outlet_Sales          8519 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.2+ KB


##Analizando datos de la columan "Oulet_Size", ya que hay una cantidad significativa de datos faltantes.

Identificando tiendas según el los datos de la columna "Oulet_Identifier"

In [28]:
#Veremos la variacion de cada tienda
weight_by_oulet_identifier = df.groupby("Outlet_Identifier")["Outlet_Size"].count()
weight_by_oulet_identifier

Outlet_Identifier
OUT010      0
OUT013    932
OUT017      0
OUT018    928
OUT019    527
OUT027    932
OUT035    930
OUT045      0
OUT046    930
OUT049    930
Name: Outlet_Size, dtype: int64

In [29]:
#Veremos la variacion de cada tienda
weight_by_outlet_q = df.groupby(["Outlet_Identifier","Outlet_Establishment_Year","Outlet_Type", "Outlet_Location_Type","Outlet_Size"])["Item_Outlet_Sales"].sum()
weight_by_outlet_q

Outlet_Identifier  Outlet_Establishment_Year  Outlet_Type        Outlet_Location_Type  Outlet_Size
OUT013             1987                       Supermarket Type1  Tier 3                High           2.142664e+06
OUT018             2009                       Supermarket Type2  Tier 3                Medium         1.851823e+06
OUT019             1985                       Grocery Store      Tier 1                Small          1.795736e+05
OUT027             1985                       Supermarket Type3  Tier 3                Medium         3.444468e+06
OUT035             2004                       Supermarket Type1  Tier 2                Small          2.268123e+06
OUT046             1997                       Supermarket Type1  Tier 1                Small          2.118395e+06
OUT049             1999                       Supermarket Type1  Tier 1                Medium         2.183970e+06
Name: Item_Outlet_Sales, dtype: float64

In [30]:
weight_by_outlet_qa = df.groupby(["Outlet_Identifier","Outlet_Establishment_Year", "Outlet_Type", "Outlet_Location_Type"])["Item_Outlet_Sales"].sum()
weight_by_outlet_qa

Outlet_Identifier  Outlet_Establishment_Year  Outlet_Type        Outlet_Location_Type
OUT010             1998                       Grocery Store      Tier 3                  1.883402e+05
OUT013             1987                       Supermarket Type1  Tier 3                  2.142664e+06
OUT017             2007                       Supermarket Type1  Tier 2                  2.167465e+06
OUT018             2009                       Supermarket Type2  Tier 3                  1.851823e+06
OUT019             1985                       Grocery Store      Tier 1                  1.795736e+05
OUT027             1985                       Supermarket Type3  Tier 3                  3.444468e+06
OUT035             2004                       Supermarket Type1  Tier 2                  2.268123e+06
OUT045             2002                       Supermarket Type1  Tier 2                  2.036725e+06
OUT046             1997                       Supermarket Type1  Tier 1                  2.118395e

Al haber mucha desinformación con respecto a los datos de la columna "Outlet_Size" y pese a que no se ha encontrado similitus con los datos evidentes, y siendo datos cualitativos, se ha decidido eliminar la columna "Outlet_Size"


In [43]:
df = df.drop(columns=["Outlet_Size"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8519 entries, 0 to 8522
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8519 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8519 non-null   object 
 3   Item_Visibility            8519 non-null   float64
 4   Item_Type                  8519 non-null   object 
 5   Item_MRP                   8519 non-null   float64
 6   Outlet_Identifier          8519 non-null   object 
 7   Outlet_Establishment_Year  8519 non-null   int64  
 8   Outlet_Location_Type       8519 non-null   object 
 9   Outlet_Type                8519 non-null   object 
 10  Item_Outlet_Sales          8519 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 798.7+ KB


In [44]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,Tier 3,Supermarket Type1,994.7052
